In [23]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

nltk.download('gutenberg')
!python3 -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/johnvillanueva/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


     |████████████████████████████████| 11.1MB 2.6MB/s eta 0:00:01
  Stored in directory: /private/var/folders/pk/swg8yj6561b2rqc8zx4vrd2m0000gn/T/pip-ephem-wheel-cache-h3yvsgof/wheels/39/ea/3b/507f7df78be8631a7a3d7090962194cf55bc1158572c0be77f
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/site-packages/en_core_web_sm -->
/usr/local/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [18]:
print(sns.__file__)

/usr/local/lib/python3.7/site-packages/seaborn/__init__.py


Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.

**Note**: Since processing all the text takes around ~5-10 minutes, in the cell below we are taking only the first tenth of each text. If you want to experiment, feel free to change the following code in the next cell:

```python
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
```
to 

```python
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)
```

In [47]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [48]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [49]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [45]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [50]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,St,thirty,croquet,wall,neat,partiality,pleasant,swim,belong,ascertain,...,autumn,guilt,solicitous,correct,sulky,voice,sensibility,wretched,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [51]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9736842105263158

Test set score: 0.8539325842696629


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [52]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(266, 3089) (266,)
Training set score: 0.9736842105263158

Test set score: 0.8707865168539326


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [58]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9774436090225563

Test set score: 0.8146067415730337


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data. Remember that for computation time concerns, we only took the first tenth of the Alice text. Emma is pretty long. **So in order to get comparable length texts, we take the first sixtieth of Emma**. Again, if you want to experiment, you can take the whole texts of each.

In [53]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma[:int(len(emma)/60)])
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [54]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [55]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [56]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 50
Processing row 100
Processing row 150
done


In [57]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.7073170731707317


col_0,Austen,Carroll
row_0,,
Austen,155,15
Carroll,57,19


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

-------------
# Challenge 0: Improving Model Accuracy

### Iteration 1: SVM

In [67]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)


print('Alice and Persuasion Texts')
print('\n',X_train.shape, y_train.shape)
print('Training set score:', svc.score(X_train, y_train))
print('Test set score:', svc.score(X_test, y_test))
svc_predicted = svc.predict(X_test)
display(pd.crosstab(y_test, svc_predicted))

print('\nAlice and Emma Texts - Model Fitted on Alice and Persuasion; Tested on Alice and Emma')
print(X_Emma_test.shape, y_Emma_test.shape)
print('Test set score:', svc.score(X_Emma_test, y_Emma_test))

svc_Emma_predicted = svc.predict(X_Emma_test)
pd.crosstab(y_Emma_test, svc_Emma_predicted)

/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Alice and Persuasion Texts

 (266, 3089) (266,)
Training set score: 0.7142857142857143
Test set score: 0.702247191011236


col_0,Austen
text_source,
Austen,125
Carroll,53



Alice and Emma Texts - Model Fitted on Alice and Persuasion; Tested on Alice and Emma
(246, 3089) (246,)
Test set score: 0.6910569105691057


col_0,Austen
row_0,
Austen,170
Carroll,76


SVC didn't work; it classified every sentence as written by Austen in both test sets with and without Emma.

### Iteration 2: Adding New SpaCy Features

Potential New Features:
- Length of the Sentence
- Noun-Phrase Chunks
- Entities

Questions:
I don't know how I would incorporate POS data into a data frame.

In [ ]:
for sent in sentences.iloc[:,0]:
    [print(ent) for ent in sent.ents]

In [ ]:
for sent in sentences.iloc[:,0]:
    [print(np) for np in sent.noun_chunks]

In [119]:
# Utility function to create a list of the 2000 most entities.
def bag_of_ents(text):
    
    # Filter out punctuation and stop words.
    allents = [ent for ent in text.ents]
    
    # Return the most common words.
    return [item[0] for item in Counter(allents).most_common(2000)]
    

# Creates a data frame with features for each entity in our common entity set.
# Each value is the count of the times the entity appears in each sentence.
def boe_features(sentences, common_ents):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_ents)
    df['text_sentence'] = sentences[0]
    #df['text_source'] = sentences[1]
    df.loc[:, common_ents] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        ents = [ent
                 for ent in sentence#.ents
                 if ent in common_ents
                 ]
        
        # Populate the row with word counts.
        for ent in ents:
            df.loc[i, ent] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
aliceents = bag_of_ents(alice_doc)
persuasionents = bag_of_ents(persuasion_doc)

# Combine bags to create a set of unique entities.
common_ents = set(aliceents + persuasionents)

In [121]:
ent_counts = boe_features(sentences, common_ents)
ent_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,"(March, 1, ,, 1760)",(first),(Alice),(two),"(four, thousand, miles)",(Elizabeth),"(a, few, minutes)",(Anne),(Anne),(Taunton),...,"(Charles, II)","(the, White, Rabbit)",(Alice),(Elizabeth),"(Mr, Wentworth)",(one),(two),"(Walter, Elliot)",(Walter),text_sentence
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind..."
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in..."
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(Oh, dear, !)"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(Oh, dear, !)"


In [116]:
sentences[0][2].ents

[Alice, Rabbit]

In [108]:
word_ent_counts = word_counts.merge(ent_counts, on='text_sentence')
word_ent_counts.shape

(444, 3608)

In [125]:
# Utility function to create a list of the 2000 most common noun_phrases.
def bag_of_nps(text):
    
    # Filter out punctuation and stop words.
    allnps = [np for np in text.noun_chunks]
    
    # Return the most common words.
    return [item[0] for item in Counter(allnps).most_common(2000)]
    

# Creates a data frame with features for each entity in our common entity set.
# Each value is the count of the times the entity appears in each sentence.
def bonp_features(sentences, common_nps):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_nps)
    df['text_sentence'] = sentences[0]
    #df['text_source'] = sentences[1]
    df.loc[:, common_nps] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        nps = [np
                 for np in sentence
                 if np in common_nps
                 ]
        
        # Populate the row with word counts.
        for np in nps:
            df.loc[i, ent] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicenps = bag_of_nps(alice_doc)
persuasionnps = bag_of_nps(persuasion_doc)

# Combine bags to create a set of unique entities.
common_nps = set(alicenps + persuasionnps)

In [123]:
np_counts = bonp_features(sentences, common_nps)
np_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,"(Charles, Musgrove)",(Shepherd),"(any, time)",(I),(it),"(the, shelves)",(she),(her),(Alice),"(all, the, warm, months)",...,(She),"(any, other, page)","(any, other, set)",(I),(wrinkles),(he),"(Lady, Russell)",(explanation),"(the, roof)",text_sentence
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind..."
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in..."
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(Oh, dear, !)"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(Oh, dear, !)"


In [126]:
word_ent_np_counts = word_ent_counts.merge(np_counts, on='text_sentence')
word_ent_np_counts.shape

(444, 6308)

In [150]:
Y = word_ent_np_counts['text_source']
X = np.array(word_ent_np_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

lr = LogisticRegression() # No need to specify l2 as it's the default. But we put it for demonstration.
lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(266, 6306) (266,)
Training set score: 0.9736842105263158

Test set score: 0.8707865168539326


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [151]:
features = word_ent_np_counts.columns.drop(['text_sentence', 'text_source'])

def boeverything_features(sentences, features):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=features)
    df['text_sentence'] = sentences[0]
    #df['text_source'] = sentences[1]
    df.loc[:, features] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        tokens = [token
                 for token in sentence
                 if token in features
                 ]
        
        # Populate the row with word counts.
        for token in tokens:
            df.loc[i, token] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

emma_everything_counts = boeverything_features(emma_sentences, features)

Processing row 0
Processing row 50
Processing row 100
Processing row 150


In [152]:
emma_everything_counts.shape

(170, 6307)

In [153]:
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_everything_counts.drop(['text_sentence'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_everything_counts.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.7682926829268293


col_0,Austen,Carroll
row_0,,
Austen,170,0
Carroll,57,19


Adding in the entities and nounphrases as features for each sentence did not help the model's accuracy when tested on alice and persuasion texts. However, applying the trained model to the alice and Emma texts improved the accuracy score by roughly 6 percent; all Emma-Austen sentence were correctly labeled as Austen. Howeverm the same number of Alice-Caroll sentences were labeld as written by Austen.

### Iteration 3: Logistic Regression with POS Tagging

Here we will run logistic regression while including features for POS tagging for the words. The original words without POS tagging are included in the final data set for modeling, along with noun phrases and entities.

In [154]:
# Utility function to create a list of the 2000 most common words.
def bag_of_tags(text):
    
    # Filter out punctuation and stop words.
    alltags = [(token.lemma_ , token.tag_)
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(alltags).most_common(2000)]

alicetags = bag_of_tags(alice_doc)
persuasiontags = bag_of_tags(persuasion_doc)

# Combine bags to create a set of unique words.
common_tags = set(alicetags + persuasiontags)

In [ ]:
[tag[0]+'_'+tag[1] for tag in common_tags]

In [164]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bot_features(sentences, common_tags):
    
    # Scaffold the data frame and initialize counts to zero.
    columns = [tag[0]+'_'+tag[1] for tag in common_tags]
    df = pd.DataFrame(columns=columns)
    df['text_sentence'] = sentences[0]
    #df['text_source'] = sentences[1]
    df.loc[:, columns] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        tags = [(token.lemma_, token.tag_)
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and (token.lemma_, token.tag_) in common_tags
                 )]
        
        # Populate the row with word counts.
        for tag in tags:
            tag_label = tag[0]+'_'+tag[1]
            df.loc[i, tag_label] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

In [165]:
tag_counts = bot_features(sentences, common_tags)
tag_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,offspring_NN,play_VBG,little_RB,easy_JJ,London_NNP,quieter_NN,note_NN,notice_NN,forget_VBN,paragraph_NN,...,state_NN,tear_VBN,pick_VBG,dab_NN,remembrance_NN,attach_VB,home_NN,dislike_VBD,invite_VBN,text_sentence
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind..."
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in..."
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(Oh, dear, !)"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(Oh, dear, !)"


In [166]:
tag_counts.shape

(444, 2084)

In [174]:
wenpt_counts = word_ent_np_counts.merge(tag_counts, on='text_sentence')
wenpt_counts.shape

(444, 8391)

In [170]:
#Creating Training set Bag of Tags on Emma Text data, then merging tag data with other features of Emma
emma_tag_counts = bot_features(emma_sentences, common_tags)
emma_wenpt_counts = emma_everything_counts.merge(emma_tag_counts, on='text_sentence')
emma_wenpt_counts.shape

Processing row 0
Processing row 50
Processing row 100
Processing row 150


(170, 8390)

In [175]:
Y = wenpt_counts['text_source']
X = np.array(wenpt_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_wenpt_counts.drop(['text_sentence'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_wenpt_counts.shape[0])])

In [176]:
#Fitting and testing the Alice vs. Caroll data with the new POS features.
lr = LogisticRegression() # No need to specify l2 as it's the default. But we put it for demonstration.
lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training Set Score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(266, 8389) (266,)
Training set score: 0.981203007518797

Test set score: 0.8707865168539326


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Nothing ever changes... (The accuracy did not increase with the new POS tagged tokens)

In [178]:
# testing model on Emma Sentence Data
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.7642276422764228


col_0,Austen,Carroll
row_0,,
Austen,169,1
Carroll,57,19


This model did slightly worse than without the POS features. So it definitely didn't do any better.

# Challenge 1: Testing the New Model -- Original Texts vs. New Text
Using only the attribute of the Caroll and Austen texts to create model features, how well can the model identify work of another author?

We will train and create features based on the Alice and Persuasion texts (particularly those features abstracted as common_words, common_nps, commons_ents, and common_tags), and use those to process the texts and train the texts. Three models with these same features will be developed, one for each previous text vs. a new text. The models will 

There is also the possibility of testing the model purely on the orginal modeling method in Challenge 0 and the associated tutorial for this drill, whereby the model is trained only on the Alice and Persuasion texts, but tested on every other text. We will not be doing that.

In [202]:
# Loading and Pre-cleaning a new text
macbeth = gutenberg.raw('shakespeare-macbeth.txt')
macbeth = re.sub(r'Chapter \d+', '', macbeth)
macbeth = text_cleaner(macbeth[:int(len(macbeth)/8)])

In [207]:
#Loading and cleaning Macbeth Text. Author is labeled as 'Other' for the sake of the drill.
macbeth_doc = nlp(macbeth)
macbeth_sents = [[sent, "Other"] for sent in macbeth_doc.sents]
macbeth_sentences = pd.DataFrame(macbeth_sents)

Bags formed from Alice and Persuasion texts: Entities, Words, Noun_Phrases, POS Tags
Create dataframes with these features for each text, then combine macbeth with alice, persuasion, and emma to create three different dataframes. These dfs will then be used to both train and test the model.

In [205]:
#Bag of Bags Features
def bob_features(sentences):
    w_counts = bow_features(sentences, common_words)
    e_counts = boe_features(sentences, common_ents)
    n_counts = bonp_features(sentences, common_nps)
    t_counts = bot_features(sentences, common_tags)
    df = w_counts.merge(e_counts, on='text_sentence')
    df = df.merge(n_counts, on='text_sentence')
    df = df.merge(t_counts, on='text_sentence')
    return df

In [ ]:
#Creating BoB counts for four texts
alice_counts = bob_features(pd.DataFrame(alice_sents))
persuasion_counts = bob_features(pd.DataFrame(persuasion_sents))
emma_counts = bob_features(emma_sentences)
macbeth_counts = bob_features(macbeth_sentences)

### Alice vs. Other

In [222]:
#Combining Alice and Macbeth BoB's, and splitting data into training and test sets.
df1 = pd.concat([alice_counts, macbeth_counts])
X = df1.loc[:, ~(df1.columns).isin(['text_sentence','text_source'])]
Y = df1['text_source']

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

lr = LogisticRegression() # No need to specify l2 as it's the default. But we put it for demonstration.
lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
lr_predicted = lr.predict(X_test)
pd.crosstab(y_test, lr_predicted)

(225, 8389) (225,)
Training set score: 0.96

Test set score: 0.847682119205298


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


col_0,Carroll,Other
text_source,,
Carroll,29,23
Other,0,99


### Persuasion vs. Other

In [223]:
df1 = pd.concat([persuasion_counts, macbeth_counts])
X = df1.loc[:, ~(df1.columns).isin(['text_sentence','text_source'])]
Y = df1['text_source']

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

lr = LogisticRegression() # No need to specify l2 as it's the default. But we put it for demonstration.
lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
lr_predicted = lr.predict(X_test)
pd.crosstab(y_test, lr_predicted)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(337, 8389) (337,)
Training set score: 0.9584569732937686

Test set score: 0.7955555555555556


col_0,Austen,Other
text_source,,
Austen,82,44
Other,2,97


### Emma vs. Other

In [224]:
df1 = pd.concat([emma_counts, macbeth_counts])
X = df1.loc[:, ~(df1.columns).isin(['text_sentence','text_source'])]
Y = df1['text_source']

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

lr = LogisticRegression() # No need to specify l2 as it's the default. But we put it for demonstration.
lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
lr_predicted = lr.predict(X_test)
pd.crosstab(y_test, lr_predicted)

(250, 8389) (250,)
Training set score: 0.92

Test set score: 0.7305389221556886


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


col_0,Austen,Other
text_source,,
Austen,29,43
Other,2,93


It's interesting that in the case of Macbeth, the model identifies nearly all of the Macbeth senteces correctly when text alongside each of the other texts. However, the macbeth does seem to have a strong coersion on labeling, because the other texts that it is compared to have many of it's other sentences labeled as Macbeth sentences. However, the error rate for mislabelling the original texts as Macbeth (Other) is not consistent. Overall, the model is fairly good at detecting whether it is Caroll, Austen, or another text.

The model did have the most trouble with the Emma vs. Macbeth texts, because the features were not created using the either of these texts.